#1. Intro


In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://gist.githubusercontent.com/flackend/18014f35d32b37c595b138f666b2108f/raw/99494b71652af807e77560b1d83ebbc5ed4c2f32/sorcerers-stone.txt

--2024-03-28 23:01:16--  https://gist.githubusercontent.com/flackend/18014f35d32b37c595b138f666b2108f/raw/99494b71652af807e77560b1d83ebbc5ed4c2f32/sorcerers-stone.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459564 (449K) [text/plain]
Saving to: ‘sorcerers-stone.txt’

sorcerers-stone.txt 100%[===================>] 448.79K  --.-KB/s    in 0.04s   

2024-03-28 23:01:16 (11.9 MB/s) - ‘sorcerers-stone.txt’ saved [459564/459564]



#2. Reading and exploring the data

In [ ]:
# read it in to inspect it
with open('sorcerers-stone.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  441832


In [ ]:
#let's look at the first 1000 characters
print(text[:1000])

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive,
were proud to say that they were perfectly normal,
thank you very much. They were the last people you’d
expect to be involved in anything strange or
mysterious, because they just didn’t hold with such
nonsense.

Mr. Dursley was the director of a firm called
Grunnings, which made drills. He was a big, beefy
man with hardly any neck, although he did have a
very large mustache. Mrs. Dursley was thin and
blonde and had nearly twice the usual amount of
neck, which came in very useful as she spent so
much of her time craning over garden fences, spying
on the neighbors. The Dursley s had a small son
called Dudley and in their opinion there was no finer
boy anywhere.

The Dursleys had everything they wanted, but they
also had a secret, and their greatest fear was that
somebody would discover it. They didn’t think they
could bear it if anyone found out about the Potters.
Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t

#3. Tokenization and train/val split


1. here basically all the characters are sorted and then assigned a number for each one of them.
2. then each of the character is then encodeded to a number till whole dataset is converted into a string of numbers
3. then it is broke into training and testing dataset
4. After that a neuron is

In [ ]:
#here are all the unique characters that occur in this text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZ\abcdefghijklmnopqrstuvwxyz—‘’“”•■
83


In [ ]:
#create a mapping from characters to integers

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder : take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[57, 58, 1, 69, 57, 54, 67, 54]
hi there


In [ ]:
# let's now encode the entire text dataset and store it into a'torch.Tensor
import torch # we use PyTorch: http://pytorch.org
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([441832]) torch.int64
tensor([43, 31, 28,  1, 25, 38, 47,  1, 46, 31, 38,  1, 35, 32, 45, 28, 27,  0,
         0, 36, 67,  9,  1, 50, 63, 53,  1, 36, 67, 68,  9,  1, 27, 70, 67, 68,
        61, 54, 74,  7,  1, 64, 55,  1, 63, 70, 62, 51, 54, 67,  1, 55, 64, 70,
        67,  7,  1, 39, 67, 58, 71, 54, 69,  1, 27, 67, 58, 71, 54,  7,  0, 72,
        54, 67, 54,  1, 65, 67, 64, 70, 53,  1, 69, 64,  1, 68, 50, 74,  1, 69,
        57, 50, 69,  1, 69, 57, 54, 74,  1, 72, 54, 67, 54,  1, 65, 54, 67, 55,
        54, 52, 69, 61, 74,  1, 63, 64, 67, 62, 50, 61,  7,  0, 69, 57, 50, 63,
        60,  1, 74, 64, 70,  1, 71, 54, 67, 74,  1, 62, 70, 52, 57,  9,  1, 43,
        57, 54, 74,  1, 72, 54, 67, 54,  1, 69, 57, 54,  1, 61, 50, 68, 69,  1,
        65, 54, 64, 65, 61, 54,  1, 74, 64, 70, 78, 53,  0, 54, 73, 65, 54, 52,
        69,  1, 69, 64,  1, 51, 54,  1, 58, 63, 71, 64, 61, 71, 54, 53,  1, 58,
        63,  1, 50, 63, 74, 69, 57, 58, 63, 56,  1, 68, 69, 67, 50, 63, 56, 54,
       

In [ ]:
#Let's now split up the data into train and validation sets

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size + 1]

tensor([43, 31, 28,  1, 25, 38, 47,  1, 46])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([43]) the target: 31
when input is tensor([43, 31]) the target: 28
when input is tensor([43, 31, 28]) the target: 1
when input is tensor([43, 31, 28,  1]) the target: 25
when input is tensor([43, 31, 28,  1, 25]) the target: 38
when input is tensor([43, 31, 28,  1, 25, 38]) the target: 47
when input is tensor([43, 31, 28,  1, 25, 38, 47]) the target: 1
when input is tensor([43, 31, 28,  1, 25, 38, 47,  1]) the target: 46


#4. Data loader : batches of chunks of data

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i : i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")



inputs:
torch.Size([4, 8])
tensor([[55,  1, 69, 57, 54,  0, 27, 70],
        [53,  1, 58, 69,  1, 64, 63, 69],
        [ 1, 65, 64, 58, 63, 69, 68,  1],
        [ 1, 64, 70, 69,  1, 58, 63, 69]])
targets:
torch.Size([4, 8])
tensor([[ 1, 69, 57, 54,  0, 27, 70, 67],
        [ 1, 58, 69,  1, 64, 63, 69, 64],
        [65, 64, 58, 63, 69, 68,  1, 55],
        [64, 70, 69,  1, 58, 63, 69, 64]])
-----
when input is [55] the target: 1
when input is [55, 1] the target: 69
when input is [55, 1, 69] the target: 57
when input is [55, 1, 69, 57] the target: 54
when input is [55, 1, 69, 57, 54] the target: 0
when input is [55, 1, 69, 57, 54, 0] the target: 27
when input is [55, 1, 69, 57, 54, 0, 27] the target: 70
when input is [55, 1, 69, 57, 54, 0, 27, 70] the target: 67
when input is [53] the target: 1
when input is [53, 1] the target: 58
when input is [53, 1, 58] the target: 69
when input is [53, 1, 58, 69] the target: 1
when input is [53, 1, 58, 69, 1] the target: 64
when input is [53, 1, 58, 

In [ ]:
print(xb)

tensor([[55,  1, 69, 57, 54,  0, 27, 70],
        [53,  1, 58, 69,  1, 64, 63, 69],
        [ 1, 65, 64, 58, 63, 69, 68,  1],
        [ 1, 64, 70, 69,  1, 58, 63, 69]])


#5. Simplest baseline: bigram language model, loss, generation



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 83])
tensor(4.3570, grad_fn=<NllLossBackward0>)

44;5’L,OB)!,2W/()7fd Ckxae"jR!Dtre0qUG“kB5(;QW•N
G—oAhhv"SsGgj4yF-H9q:Ug■jgDJ4S11Bv!6Ulpnlitw;h’()KP


#6. Training the Bigram Model


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.714195728302002


#7. Port our code to a script

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


)T
UT6t•-!K?n-Jqn3nIrtE\MY,3nb849Gy'hyNb1!?ZWlElwlwp■Lgnsod‘,J"4E/o'Z,lPA8cT‘QJ;l4GFll:qL!hztmoT!RJtKPxx:cJ!aCnm-H4Pxbnw?- k!ERFk"y0n-(!NkWYK7hC2NY■xO75TMB;6,Z'Lz C
egMj2pf—sOgrYW9'—M l4•12"yF6orjlaO.”y'5RSEQ7ZLr6z2 k‘WIkPl0l(lC
lW928?7s0a“pntsESg?5■‘n6OFaKao“UL’rLP;;n-oJjOESGsVT,JgsJl-RCh!7IJwZ
akSn
:xSW-J.uS”Vq)nh:e0■DZ■I’e,P)d
w‘nGpmx, WeZW(BR)"2Yj7vLS)TFy‘”tm N
5AbkFj•k.,ZKy3FKH49RRMOYMs Na“p8Nx8NI3”■M?OBzK84V”tY'—ISuqGsfC■1FJHQ5W/Pz.LrKsz):Y5vor5K0nt84s”:x‘/gN3ntBR"1qT2NwZnx!vKZE3N.w6r?f?T 


#8. The trick in self-attention: matrix multiply as weight aggregation

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

#9. V2: Using matrix multiplication

In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

#10. V3. Adding softmax

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

#12. V4. self attention

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

#13. n1: attention as communication


Notes:

1. Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
2. There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
3. Each example across batch dimension is of course processed completely independently and never "talk" to each other
4.In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
5. "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
6. "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

#14. Inserting a single self-attention block to our network

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(0.9831)

In [ ]:
q.var()

tensor(1.0026)

In [ ]:
wei.var()

tensor(1.0456)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



#

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('sorcerers-stone.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


0.212051 M parameters
step 0: train loss 4.6292, val loss 4.6379
step 100: train loss 2.6828, val loss 2.6865
step 200: train loss 2.5206, val loss 2.5135
step 300: train loss 2.4815, val loss 2.4703
step 400: train loss 2.3916, val loss 2.3789
step 500: train loss 2.2997, val loss 2.2922
step 600: train loss 2.2432, val loss 2.2296
step 700: train loss 2.1636, val loss 2.1628
step 800: train loss 2.1059, val loss 2.1173
step 900: train loss 2.0570, val loss 2.0616
step 1000: train loss 2.0136, val loss 2.0154
step 1100: train loss 1.9834, val loss 2.0010
step 1200: train loss 1.9532, val loss 1.9684
step 1300: train loss 1.9211, val loss 1.9338
step 1400: train loss 1.9033, val loss 1.9226
step 1500: train loss 1.8629, val loss 1.8875
step 1600: train loss 1.8461, val loss 1.8589
step 1700: train loss 1.8239, val loss 1.8645
step 1800: train loss 1.8100, val loss 1.8295
step 1900: train loss 1.7784, val loss 1.7940
step 2000: train loss 1.7735, val loss 1.7970
step 2100: train loss 1.

#15. Saving model

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/saved_models/modelboy1.h5')


#16. Prompting and output

In [ ]:
model = BigramLanguageModel()
state_dict = torch.load('/content/drive/MyDrive/saved_models/modelboy1.h5')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

# Tokenize the starting prompt
prompt = "what is the difference between harry and hagrid"
tokens = torch.tensor(encode(prompt), dtype=torch.long).unsqueeze(0)


# Generate the conversation
max_tokens = 2000
word_count = 0

  # Maximum number of tokens to generate
for _ in range(max_tokens):
    # Generate the next token
    with torch.no_grad():
        output = model.generate(tokens, max_new_tokens=1)
    new_token = output[:, -1].item()

    # Convert the new token to a character and append to the prompt
    # Convert the new token to a character and append to the prompt
    new_char = itos[new_token]
    if new_char == '':
      word_count += 1
    tokens = torch.cat((tokens, torch.tensor([[new_token]])), dim=1)

    if word_count >= 2000:
      break
    # Print the conversation so far
# Convert the token indices to characters and join them into a string
generated_text = ''.join([itos[idx] for idx in tokens.squeeze().tolist()])

# Print the conversation so far
print(generated_text)

what is the difference between harry and hagrid of her
seen in clutching the Gryffial the Restrictaker was open down it on thousand Ron manother worse.

“Could told hope?” Hagrid
shoudy are them in everything
he could ever tomor and fire left once, Uncle Verty told here, I’m
golden a ticked as choosions it. He had
reached.

“Yeah,” said Wood. “have you, taring over his
legs with hide? I was like that evelops over the match,
rubbed at made. A your gamekeen.

“About it all, but Harry?”

Harry hurried cail
way. “Answeups, and he couldn’t see her good; a great now thin’. exportictaking made something. Cheel. It don’ waits
toward, Harry much me first and clutching it, but
at last the game ookly, rapping their was
acrat.

On the planes sort’s he end oujn which admiral possible coming were
let have to steal sleeks, a Stain
jearty? But it one told tell kitchen his front he lost mind
without — ”







At liugh of the others toward Harry and
gloones could have laughed toward crack to wait, as t